# 1. Install and Import Libraries

In [0]:
%sh
pip install --quiet tqdm huggingface_hub aiohttp nest_asyncio psycopg2-binary uuid-utils


[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [0]:
import argparse
import asyncio
from dataclasses import dataclass
import datetime as dt
from itertools import islice
import json
import logging
from logging.handlers import TimedRotatingFileHandler
import nest_asyncio
import os
import random
import re
import sys
import time
from typing import Any, Optional, Union
from urllib.parse import urlparse

import aiohttp
from bs4 import BeautifulSoup
from huggingface_hub import HfApi
import pandas as pd
from pandas.api.types import is_object_dtype
import psycopg2
import pyspark
from pyspark import sql as pysql
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructType, StructField,
    StringType, ShortType, IntegerType, LongType, DoubleType, BooleanType,
    TimestampType, ArrayType, MapType
)
from pyspark.sql import functions as F
import requests
from tqdm import tqdm
from uuid_utils import uuid7

nest_asyncio.apply()

In [0]:
ENCODING_TYPE = 'utf-8-sig'
BASE = "https://huggingface.co"
HF_READ_TOKEN = "YOUR_TOKEN"
HF_WRITE_TOKEN = "YOUR_TOKEN"

# 2. Set Schema Using Pyspark StructType

## 2-1. Metadata

In [0]:
meta_schema = StructType([
    StructField("id",                   StringType(),       False), # PK, NOT NULL
    StructField("hexid",                StringType(),       True),  # CHAR(24)
    StructField("sha",                  StringType(),       True),  # CHAR(40)
    StructField("author",               StringType(),       True),  # 
    StructField("private",              BooleanType(),      True),  # 
    StructField("disabled",             BooleanType(),      True),  # 
    StructField("gated",                StringType(),       True),  #   
    StructField("pipeline_tag",         StringType(),       True),  # 
    StructField("library_name",         StringType(),       True),  # 
    StructField("tags",                 StringType(),       True),  # '["a","b"]' 같은 문자열
    StructField("likes",                DoubleType(),       True),  # 
    StructField("downloads",            DoubleType(),       True),  # 
    StructField("trending_score",       DoubleType(),       True),  # 
    StructField("created_at",           TimestampType(),    True),  # 
    StructField("last_modified",        TimestampType(),    True),  # 
    StructField("config",               StringType(),       True),  # JSON 문자열
    StructField("card_data",            StringType(),       True),  # JSON 문자열
    StructField("safetensors",          StringType(),       True),  # JSON 문자열
    StructField("spaces",               StringType(),       True),  # '["a","b"]' 같은 문자열
    StructField("transformers_info",    StringType(),       True),  # JSON 문자열
    StructField("used_storage",         DoubleType(),       True),  # 
    StructField("readme",               StringType(),       True),  # 

    StructField("model_index",          StringType(),       True),  # '["a","b"]' 같은 문자열
    StructField("gguf",                 StringType(),       True),  # 
    StructField("inference",            StringType(),       True),  # 
    StructField("mask_token",           StringType(),       True),  # 
    StructField("widget_data",          StringType(),       True),  # 
    StructField("security_repo_status", StringType(),       True)   # 
])

## 2-2. Files

In [0]:
file_schema = StructType([
    StructField("uuid7",           StringType(),    False), # PK, NOT NULL
    StructField("model_id",        StringType(),    False), # FK, NOT NULL
    StructField("type",            StringType(),    True),  # 
    StructField("oid",             StringType(),    True),  # 
    StructField("size",            LongType(),      True),  # 
    StructField("path",            StringType(),    True),  # 
    StructField("lfs_oid",         StringType(),    True),  # 
    StructField("lfs_size",        DoubleType(),    True),  # 
    StructField("lfs_pointersize", DoubleType(),    True),  # 
    StructField("xethash",         StringType(),    True)   # 
])

## 2-3. Community

In [0]:
comu_schema = StructType([
    StructField("uuid7",                        StringType(),       False), # PK, NOT NULL
    StructField("model_id",                     StringType(),       False), # FK, NOT NULL
    StructField("num",                          LongType(),         True),  # 
    StructField("title",                        StringType(),       True),  # 
    StructField("status",                       StringType(),       True),  # 
    StructField("created_at",                    TimestampType(),    True),  # 
    StructField("is_pull_request",                BooleanType(),      True),  # 
    StructField("num_comments",                  LongType(),         True),  # 
    StructField("top_reactions",                 StringType(),       True),  # JSON 문자열
    StructField("num_reaction_users",             LongType(),         True),  # 
    StructField("pinned",                       BooleanType(),      True),  # 
    StructField("author_id",                   StringType(),       True),  # CHAR(24)
    StructField("author_avatar_url",             StringType(),       True),  # 
    StructField("author_fullname",              StringType(),       True),  # 
    StructField("author_name",                  StringType(),       True),  # 
    StructField("author_type",                  StringType(),       True),  # 
    StructField("author_ispro",                 BooleanType(),      True),  # 
    StructField("author_ishf",                  BooleanType(),      True),  # 
    StructField("author_ishf_admin",             BooleanType(),      True),  # 
    StructField("author_ismod",                 BooleanType(),      True),  # 
    StructField("repo_name",                    StringType(),       True),  # 
    StructField("repo_type",                    StringType(),       True),  # 
    StructField("repo_owner_name",               StringType(),       True),  # 
    StructField("repo_owner_isparticipating",    BooleanType(),      True),  # 
    StructField("repo_owner_type",               StringType(),       True),  # 
    StructField("repo_owner_isdiscussion_author", BooleanType(),      True),  # 
    StructField("author_follower_count",         DoubleType(),       True),  # 
])

# 3. Create PostgreSQL Schema

In [0]:
SCHEMA_LEVEL = "bronze"
META_TABLE_NAME = "brz_hf_meta"
FILE_TABLE_NAME = "brz_hf_files"
COMU_TABLE_NAME = "brz_hf_community"

In [0]:
conn = psycopg2.connect(
    host="HOST",
    port="PORT",
    dbname="DB_NAME",
    user="USER_NAME",
    password="PASSWORD"
)
cur = conn.cursor()

In [0]:
# 기존 테이블 드롭 (CASCADE 포함)
cur.execute(f"DROP TABLE IF EXISTS {SCHEMA_LEVEL}.{META_TABLE_NAME} CASCADE;")
cur.execute(f"DROP TABLE IF EXISTS {SCHEMA_LEVEL}.{FILE_TABLE_NAME} CASCADE;")
cur.execute(f"DROP TABLE IF EXISTS {SCHEMA_LEVEL}.{COMU_TABLE_NAME} CASCADE;")

## 3-1. Metadata

In [0]:
cur.execute(f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_LEVEL}.{META_TABLE_NAME} (
  id                   text NOT NULL PRIMARY KEY,
  hexid                text,
  sha                  text,
  author               text,
  private              boolean,
  disabled             boolean,
  gated                text,
  pipeline_tag         text,
  library_name         text,
  tags                 text ,
  likes                double precision,
  downloads            double precision,
  trending_score       double precision,
  created_at           timestamptz,
  last_modified        timestamptz,
  config               text,
  card_data            text,
  safetensors          text,
  spaces               text,
  transformers_info    text,
  used_storage         double precision,
  readme               text,
  model_index          text,
  gguf                 text,
  inference            text,
  mask_token           text,
  widget_data          text,
  security_repo_status text
);
""")

## 3-2. Files

In [0]:
cur.execute(f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_LEVEL}.{FILE_TABLE_NAME} (
    uuid7           TEXT NOT NULL PRIMARY KEY,
    model_id        TEXT NOT NULL,
    type            TEXT,
    oid             TEXT,
    size            BIGINT,
    path            TEXT,
    lfs_oid         TEXT,
    lfs_size        DOUBLE PRECISION,
    lfs_pointersize DOUBLE PRECISION,
    xethash         TEXT
);
""")

# -- FK 정의 (모델 메타 테이블과 연결)
#     CONSTRAINT fk_hfmeta
#         FOREIGN KEY (model_id)
#         REFERENCES {SCHEMA_LEVEL}.{META_TABLE_NAME} (id)

## 3-3. Community

In [0]:
cur.execute(f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_LEVEL}.{COMU_TABLE_NAME} (
    uuid7                           TEXT NOT NULL PRIMARY KEY,
    model_id                        TEXT NOT NULL,
    num                             BIGINT,
    title                           TEXT,
    status                          TEXT,
    created_at                       TIMESTAMPTZ,
    is_pull_request                   BOOLEAN,
    num_comments                     BIGINT,
    top_reactions                    TEXT,
    num_reaction_users                BIGINT,
    pinned                          BOOLEAN,
    author_id                      TEXT,
    author_avatar_url                TEXT,
    author_fullname                 TEXT,
    author_name                     TEXT,
    author_type                     TEXT,
    author_ispro                    BOOLEAN,
    author_ishf                     BOOLEAN,
    author_ishf_admin                BOOLEAN,
    author_ismod                    BOOLEAN,
    repo_name                       TEXT,
    repo_type                       TEXT,
    repo_owner_name                  TEXT,
    repo_owner_isparticipating       BOOLEAN,
    repo_owner_type                  TEXT,
    repo_owner_isdiscussion_author    BOOLEAN,
    author_follower_count            DOUBLE PRECISION    
);
""")

# -- FK 정의 (모델 메타 테이블과 연결)
#     CONSTRAINT fk_hfmeta
#         FOREIGN KEY (model_id)
#         REFERENCES {SCHEMA_LEVEL}.{META_TABLE_NAME} (id)

In [0]:
conn.commit()
cur.close()
conn.close()

---------------------------------------------------------------------------
InterfaceError                            Traceback (most recent call last)
File <command-5091735938707682>, line 1
----> 1 conn.commit()
      2 cur.close()
      3 conn.close()

InterfaceError: connection already closed

# 4. INSERT data

In [0]:
def insert_data(
    data: Union[pd.DataFrame, pyspark.sql.DataFrame],
    schema: str,
    table: str,
    mode: str = "overwrite"
) -> None:
    # spark 데이터프레임 생성
    columns = list(data.columns)
    if isinstance(data, pd.DataFrame):
        # data = spark.createDataFrame(data, schema=columns)
        data = spark.createDataFrame(data)
    
    # 인증 정보 하드코딩
    jdbc_user = "USER_NAME"
    jdbc_pass = "PASSWORD"

    # 연결 문자열 하드코딩
    host = "HOST"
    port = "PORT"
    jdbc_url = f"jdbc:postgresql://{host}:{port}/postgres"
    table_lc = table.lower()

    if mode == "upsert":
        staging = f"{table_lc}_stg"

        # 1) 스테이징에 덮어쓰기 적재
        data.write.format("jdbc").mode("overwrite").options(
            url=jdbc_url,
            user=jdbc_user,
            password=jdbc_pass,
            dbtable=f"{schema}.{staging}",
            truncate=True,      # 데이터만 비움
            batchsize=1000
        ).save()

        # 2) ON CONFLICT 업서트
        cols = [f'"{c}"' for c in data.columns]
        col_list = ", ".join(cols)
        key_list = ", ".join([f'"{k}"' for k in ("uuid7",)])
        update_cols = [c for c in data.columns if c not in ("uuid7",)]
        set_clause = ", ".join([f'"{c}"=EXCLUDED."{c}"' for c in update_cols]) or "NOTHING"

        sql = f"""
        INSERT INTO {schema}.{table_lc} ({col_list})
        SELECT {col_list} FROM {schema}.{staging}
        ON CONFLICT ({key_list}) DO UPDATE SET
        {set_clause};
        """

        conn = psycopg2.connect(
            host="HOST",
            port="PORT",
            dbname="postgres",
            user=jdbc_user,
            password=jdbc_pass
        )
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            cur.execute(f'DROP TABLE IF EXISTS {schema}.{staging};')
        conn.close()

    else:
        data.write.format("jdbc").mode(mode).options(
            url=jdbc_url,
            user=jdbc_user,
            password=jdbc_pass,

            # 가끔 테이블 이름을 대소문자 혼용하여 인자로 던져주는 경우를 대비한 코드
            # PostgreSQL의 테이블은 언제나 소문자만 사용함
            dbtable=f"{schema}.{table_lc}",

            # overwrite일 때 DROP 후 CREATE가 아니라, TRUNCATE TABLE로 데이터만 비움
            # truncate=True,

            # 대량 적재 튜닝: INSERT할 때 한 번에 몇 행씩 묶어서 넣을지
            batchsize=1000,

            # 스키마 적재 튜닝: INSERT시 JDBC는 모든 문자열 컬럼에 대해 setString() 함수를 사용해서, 자동으로 PosgreSQL에 text로 적재함
            # 이 옵션을 사용하면 unspecified로 넘겨 PostgreSQL 테이블의 스키마를 확인하고, 해당 스키마에 맞게끔 캐스팅함
            # 대신 넣으려는 데이터와 테이블의 스키마를 정확히 일치시켜야 함. 그렇지 않으면 에러가 발생함
            # 에러가 발생하는 경우 그냥 주석처리하거나, PostgreSQL 테이블 스키마를 TEXT로 전부 바꾸면 해결
            # stringtype="unspecified"
        ).save()

## 4-1. Metadata

In [0]:
# tmp_df = pd.read_csv('hf_metadata.csv', encoding=ENCODING_TYPE)
# tmp_df["created_at"] = pd.to_datetime(tmp_df["created_at"], utc=True, errors="coerce")
# tmp_df["last_modified"] = pd.to_datetime(tmp_df["last_modified"], utc=True, errors="coerce")
# # tmp_df["model_index"] = tmp_df["model_index"].map(lambda x: list(x) if isinstance(x, str) else x)
# tmp_df["likes"] = tmp_df["likes"].fillna(0).astype("int64")
# tmp_df["downloads"] = tmp_df["downloads"].fillna(0).astype("int64")
# tmp_df["used_storage"] = tmp_df["used_storage"].fillna(0).astype("int64")
# for c in tmp_df.columns:
#     tmp_df[c] = tmp_df[c].where(pd.notna(tmp_df[c]), None)

# # 특정 테이블에 데이터를 삽입하는 코드
# # mode 종류: overwrite, append, ignore, error
# insert_data(
#     data=spark.createDataFrame(tmp_df, schema=meta_schema),
#     schema=SCHEMA_LEVEL,
#     table=META_TABLE_NAME,
#     mode="overwrite"
# )

for i in tqdm(range(1, 21 + 1)):
    tmp_df = pd.read_csv(f'split/hf_metadata_{i}.csv', encoding=ENCODING_TYPE, low_memory=False).drop_duplicates(subset=['id'])
    tmp_df["created_at"] = pd.to_datetime(tmp_df["created_at"], utc=True, errors="coerce")
    tmp_df["last_modified"] = pd.to_datetime(tmp_df["last_modified"], utc=True, errors="coerce")

    # 특정 테이블에 데이터를 삽입하는 코드
    # mode 종류: overwrite, append, ignore, error, upsert(user definition)
    insert_data(
        data=spark.createDataFrame(tmp_df, schema=meta_schema),
        schema=SCHEMA_LEVEL,
        table=META_TABLE_NAME,
        mode="upsert"
    )

100%|██████████| 2/2 [02:12<00:00, 66.43s/it]


## 4-2. Files

In [0]:
for i in tqdm(range(1, 168 + 1)):
    tmp_df = pd.read_csv(f'split/hf_files_{i}.csv', encoding=ENCODING_TYPE, low_memory=False)
    tmp_df = tmp_df[tmp_df['path'].notna()]

    tmp_df["uuid7"] = [str(uuid7()) for _ in range(len(tmp_df))]
    cols = ["uuid7"] + [c for c in tmp_df.columns if c != "uuid7"]
    tmp_df = tmp_df[cols].rename(columns={"lfs_pointerSize": "lfs_pointersize", "xetHash": "xethash"})

    # 특정 테이블에 데이터를 삽입하는 코드
    # mode 종류: overwrite, append, ignore, error
    insert_data(
        data=spark.createDataFrame(tmp_df, schema=file_schema),
        schema=SCHEMA_LEVEL,
        table=FILE_TABLE_NAME,
        mode="append"
    )

## 4-3. Community

In [0]:
def preprocessing(tmp_df: pd.DataFrame) -> pd.DataFrame:
    tmp_df["created_at"] = pd.to_datetime(tmp_df["created_at"], utc=True, errors="coerce")
    tmp_df[["author_ispro", "author_ishf", "author_ishf_admin", "author_ismod"]] = \
        tmp_df[["author_ispro", "author_ishf", "author_ishf_admin", "author_ismod"]].fillna(False).astype(bool)
    tmp_df["uuid7"] = [str(uuid7()) for _ in range(len(tmp_df))]

    # before_cols = tmp_df.columns.tolist()
    # after_cols = [
    #     "uuid", "model_id", "num", "title", "status" ,"created_at", "is_pull_request", "num_comments", "top_reactions", 'num_reaction_users' ,"pinned",
    #     "author_id", "author_avatar_url", "author_fullname", "author_name", "author_type", "author_ispro", "author_ishf", "author_ishf_admin", "author_ismod", 
    #     "author_follower_count",
    #     "repo_name", "repo_type", "repo_owner_name", "repo_owner_isparticipating", "repo_owner_type", "repo_owner_isdiscussion_author",
    # ]
    # rename_dict = dict(zip(before_cols, after_cols))
    # tmp_df.rename(columns=rename_dict, inplace=True)

    reindex_cols = [
        "uuid7", "model_id", "num", "title", "status" ,"created_at", "is_pull_request", "num_comments", "top_reactions", 'num_reaction_users' ,"pinned",
        "author_id", "author_avatar_url", "author_fullname", "author_name", "author_type", "author_ispro", "author_ishf", "author_ishf_admin", "author_ismod", 
        "repo_name", "repo_type", "repo_owner_name", "repo_owner_isparticipating", "repo_owner_type", "repo_owner_isdiscussion_author",
        "author_follower_count",
    ]
    tmp_df = tmp_df.reindex(columns=reindex_cols).where(pd.notna(tmp_df), None)
    return tmp_df

In [0]:
for i in tqdm(range(1, 4 + 1)):
    tmp_df = pd.read_csv(f'split/hf_community_{i}.csv', encoding=ENCODING_TYPE, low_memory=False)
    tmp_df = preprocessing(tmp_df)

    # 특정 테이블에 데이터를 삽입하는 코드
    # mode 종류: overwrite, append, ignore, error
    insert_data(
        data=spark.createDataFrame(tmp_df, schema=comu_schema),
        schema=SCHEMA_LEVEL,
        table=COMU_TABLE_NAME,
        mode="append"
    )

100%|██████████| 4/4 [00:39<00:00,  9.89s/it]
